In [ ]:
# To mount
from google.colab import drive
drive.mount('/content/drive')

# # To initialize the work environment
# %cd /content/drive/My Drive/
# !git clone https://github.com/allnightlight/SIDwithIS/casestudies -b casestudies

# # To update the work environment
# %cd /content/drive/My Drive/SIDwithIS
# !git pull

%cd /content/drive/My Drive/SIDwithIS/casestudies

In [ ]:
import os
import sys
import numpy as np
sys.path.append("../framework/")
sys.path.append("../sl/")
sys.path.append("../sid/")

from builder import Builder
from loader import Loader
from mylogger import MyLogger
from sid_agent import SidAgent
from sid_agent_factory import SidAgentFactory
from sid_build_parameter import SidBuildParameter
from sid_trainer_factory import SidTrainerFactory
from store import Store
from sid_evaluator import SidEvaluator
from builtins import isinstance
from cs02_environment_factory import Cs02EnvironmentFactory
from cs02_build_parameter_factory import Cs02BuildParameterFactory
from cs02_build_parameter import Cs02BuildParameter

In [ ]:
trainingHistoryDbPath = "cs02_training_history.sqlite"

In [ ]:
target_casestudy = "cs02a-01"

## S100: Create a test data set:

In [ ]:
def generateTestData(dataFilePath):

    Nsample = 2**12
    Nu = 2
    Ny = 3

    Timestamp = np.arange(Nsample)
    U = np.random.randn(Nsample, Nu)
    Y = np.random.randn(Nsample, Ny)
    Ev = np.random.randint(2, size=(Nsample,))

    Y[np.random.rand(Nsample, Ny) < 1/2**5] = np.nan
    U[np.random.rand(Nsample, Nu) < 1/2**5] = np.nan

    tbl = []
    header = ["timestamp", "event",] + ["U%d" % k1 for k1 in range(Nu)] + ["Y%d" % k1 for k1 in range(Ny)]
    tbl.append(header)
    for seg, ev, u, y in zip(Timestamp, Ev.astype(np.int), U, Y):    
        row = [seg, ev, *u, *y]
        tbl.append(row)

    with open(dataFilePath, "w") as fp:
        fp.write(",".join(header)+"\n")
        for line in tbl[1:]:
            fp.write(",".join([str(elm) for elm in line])+"\n")

In [ ]:
dataFilePath = "testData.csv"
generateTestData(dataFilePath)

## S300: Run learning agents

### SS310: initialize an instance of builder

In [ ]:
agentFactory = SidAgentFactory()
environmentFactory = Cs02EnvironmentFactory()
trainerFactory = SidTrainerFactory()

logger = MyLogger(console_print=True)
store = Store(trainingHistoryDbPath)

builder = Builder(agentFactory=agentFactory
                  , environmentFactory=environmentFactory
                  , trainerFactory=trainerFactory
                  , store=store
                  , logger = logger)

### SS312: define prediction horizon

In [ ]:
N1 = 2**4 # prediction horizon

### SS320: build naive agents:

In [ ]:
buildParameter = Cs02BuildParameter(
    nEpoch= 0
    , label= target_casestudy 
    , agentClass="agent_naive"
    , N1 = N1
    , Ntrain=2 ** 11
    , dataFilePath=dataFilePath)

builder.build(buildParameter)

### SS330: run build

In [ ]:
cnt = 1
nEpoch = 2**7

while True:
    
    agentClass = np.random.choice(("agent001",))
    N0 = int(np.random.choice((1, 2**2, 2**4)))
    N1 = N1
    NhiddenAgent = int(np.random.choice((2**1, 2**2, 2**4)))
    sampling_balance = float(np.random.choice((0.1, 0.5, 0.9)))
    use_offset_compensate = int(np.random.choice((0, 1)))
    use_imbalanced_sampling = int(np.random.choice((0, 1)))
    seed = int(np.random.randint(5))
    
    buildParameter = Cs02BuildParameter(
        nIntervalSave= nEpoch//2
        , nEpoch= nEpoch
        , label= target_casestudy + " %3d" % cnt
        , sampling_balance=sampling_balance
        , NhiddenAgent=NhiddenAgent
        , agentClass="agent001"
        , Nbatch=2 ** 6
        , N0=N0
        , N1=N1
        , Ntrain=2 ** 11
        , dampingConstantInitial=0.99
        , use_offset_compensate=use_offset_compensate
        , use_imbalanced_sampling=use_imbalanced_sampling
        , dataFilePath=dataFilePath)
    
    builder.build(buildParameter)
    cnt += 1